# Meta - Instagram Creative Post Engagement and Sharing Patterns

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_agg = pd.read_csv('Data/022/agg_daily_creative_shares.csv')
df_agg.head()

,user_id,content_type,share_count,share_date
0,101,photo,6,2024-04-05
1,101,video,7,2024-04-20
2,101,photo,5,2024-05-08
3,101,video,4,2024-05-21
4,101,photo,6,2024-06-10


# Pregunta 1

### ¿Qué usuarios compartieron fotos o videos creativos (es decir, la suma total de veces que compartieron) más de 10 veces en abril de 2024? Este análisis ayudará a determinar qué usuarios están altamente comprometidos con la compartición de contenido.

In [4]:
df_agg['share_date'] = pd.to_datetime(df_agg['share_date'])

df_abril = df_agg[
    (df_agg['share_date'].dt.year == 2024) &
    (df_agg['share_date'].dt.month == 4)
]

total_por_usuario = df_abril.groupby('user_id')['share_count'].sum().reset_index()

usuarios_top = total_por_usuario[total_por_usuario['share_count'] > 10]

usuarios_top

,user_id,share_count
0,101,13
2,103,11
3,104,11
5,106,11


```SQL
SELECT
    user_id,
    SUM(share_count) AS total_share
FROM agg_daily_creative_shares
WHERE EXTRACT (YEAR FROM share_date) = 2024
AND EXTRACT (MONTH FROM share_date) = 4
GROUP BY user_id
HAVING SUM(share_count) > 10;

# Pregunta 2

### ¿Cuál es el promedio de contenidos compartidos por usuario en mayo de 2024, entre aquellos que compartieron al menos una vez? Primero queremos obtener el total de compartidos por cada usuario en mayo de 2024 y, después, calcular el promedio sobre todos esos usuarios.

In [6]:
mayo_df = df_agg[df_agg['share_date'].between('2024-05-01','2024-05-31')]

totales_usuario = mayo_df.groupby('user_id')['share_count'].sum()

resultado_final = totales_usuario.mean()

resultado_final

np.float64(7.833333333333333)

```SQL
SELECT
    AVG(total_shares_per_user)
FROM (
    SELECT
        user_id,
        SUM(share_count) AS total_shares_per_user
    FROM agg_daily_creative_shares
    WHERE share_date >= '2024-05-01' AND share_date <= '2024-05-31'
    GROUP BY user_id
) AS monthly_totals;
```

# Pregunta 3

### Para cada usuario de Instagram que compartió contenido creativo, ¿cuál es el valor redondeado hacia abajo (floor) de su promedio de compartidos diarios durante el segundo trimestre de 2024? Incluye únicamente a los usuarios con un promedio de al menos 5 compartidos por día.

### Nota: la tabla agg_daily_creative_shares está al nivel de tipo de contenido, usuario y día. Asegúrate de agregar los datos al nivel de 'usuario-día' antes de calcular el promedio.

In [11]:
df_q2 = df_agg[df_agg['share_date'].between('2024-04-01','2024-06-30')]

df_diario = df_q2.groupby(['user_id','share_date'])['share_count'].sum().reset_index()

reporte = df_diario.groupby('user_id')['share_count'].mean().reset_index()

reporte = reporte[reporte['share_count'] >= 5]
reporte['share_count'] = reporte['share_count'].apply(np.floor)

reporte


,user_id,share_count
0,101,5.0
2,103,5.0
5,106,6.0


```SQL
SELECT 
    user_id,
    FLOOR(AVG(total_diario)) AS avg_daily_shares
FROM (
    -- Paso 1: Agregamos a nivel usuario-día
    SELECT 
        user_id, 
        share_date, 
        SUM(share_count) AS total_diario
    FROM agg_daily_creative_shares
    WHERE share_date BETWEEN '2024-04-01' AND '2024-06-30'
    GROUP BY user_id, share_date
) AS tabla_diaria
GROUP BY user_id
HAVING AVG(total_diario) >= 5;
```